# CX2 -> networkx -> remove zero degree nodes -> upload

In this tutorial we will be downloading a network from NDEx, converting that network to networkx `Graph()` object via the `get_graph()` method from `CX2NetworkXFactory`, removing 0 degree nodes, and uploading the new network back to NDEx. 

This tutorial requires Python 3.8+ and networkx 2.4+

# Import modules

Import needed modules. If there is an error with matplotlib run:

`conda install -c conda-forge matplotlib`

**NOTE:** For better support for Anaconda environments, exit Jupyter and run:
          `conda install -c anaconda nb_conda`

In [ ]:
import matplotlib

# If notebook crashes or gives matplotlib error try uncommenting the next line 
# matplotlib.use('Qt4Agg')

# this matplotlib inline enables the matplotlib plots to be displayed in the Jupyter notebook
%matplotlib inline

import matplotlib.pyplot as plt
import ndex2
from ndex2 import cx2
from ndex2.client import DecimalEncoder
import networkx
import json
import io

# regular expression library
import re

# used to prompt user for NDEx password in this notebook
import getpass


# Download the example network from NDEx

Using the `get_network_as_cx2_stream()` from `ndex2.client.Ndex2()` and `cx2.RawCX2NetworkFactory`, the following line of code downloads the network from NDEx and creates a `CX2Network` object named `cx2_network`. 

For help on function names try running `help(cx2_network)`

In [ ]:
# EXAMPLE NETWORK FROM NDEx CIViC Variant-Drug Associations
# Viewable in a browser here: https://www.ndexbio.org/viewer/networks/58b909ff-d961-11ed-b4a3-005056ae23aa
civic_variant_drug_network_uuid = '58b909ff-d961-11ed-b4a3-005056ae23aa'

# Create NDEx2 python client
client = ndex2.client.Ndex2()

# Create CX2Network factory
factory = cx2.RawCX2NetworkFactory()

# Get network as cx2 stream
client_resp = client.get_network_as_cx2_stream(civic_variant_drug_network_uuid)

# Convert downloaded network to CX2Network object
cx2_network = factory.get_cx2network(json.loads(client_resp.content))

# Get positions of nodes from CX2Netowork

This fragment of code uses `get_nodes()` on the `cx2_network` and get the positions stored in node attributes.

In [ ]:
node_positions={}

# Interate through nodes of cx2_network
for node_id, node_attr in cx2_network.get_nodes().items():
    node_positions[node_id]=(node_attr[ndex2.constants.LAYOUT_X],
                             node_attr[ndex2.constants.LAYOUT_Y])
print(len(node_positions))

# Make a networkx graph from the CX2

This fragment of code uses `to_graph()` from `cx2.CX2NetworkXFactory` to make a networkx `Graph()` object. 

In [ ]:
networkx_factory = cx2.CX2NetworkXFactory()

g = networkx_factory.get_graph(cx2_network)

print('CX2Network number of nodes: ' + str(len(cx2_network.get_nodes())) + 
      ' vs Networkx: ' + str(len(list(g))))
print('CX2Network number of edges: ' + str(len(cx2_network.get_edges())) +
      ' vs Networkx: ' + str(g.number_of_edges()))

# the networkx object has the name and description of the original network
if g.graph.get('name'):
   print(g.graph)

# Plot your network in the notebook

If the following fails, the issue might be due to installation of older version of networkx (1.11) 

To fix this, exit jupyter and run the following in the terminal:

`pip install networkx --upgrade`

Then restart jupyter notebook (ie `jupyter notebook`) 

In [ ]:
print("networkx version: " + networkx.__version__)
print("starting...")
plt.rcParams['figure.dpi'] = 200
networkx.draw_networkx(g, pos=node_positions, node_size=5, with_labels=False, width=0.2)

# Get a list of the zero degree nodes

We use the networkx method `isolates()` to get a list of all zero degree nodes and then store them in the variable `isolatelist`

In [ ]:
isolatelist = list(networkx.isolates(g))
print('Number of zero degree nodes: ' + str(len(isolatelist)))

# Remove the zero degree nodes and their positions

We will now remove the zero degree nodes by their ids on the `isolatelist`

In [ ]:
print('Number of nodes before removal: ' + str(len(list(g))))
# the nodes in the networkx graph are the node ids from the NiceCX network

g.remove_nodes_from(isolatelist)

print('Number of nodes after removal: ' + str(len(list(g))))

# Convert the networkx object back to a CX2Network object

The following code converts the networkx graph object `g` back to a 
`CX2Network` object using `NetworkXToCX2NetworkFactory` and `get_cx2network()` method.

Additional calls below set the name of the network as well as the description. 

Finally, for debugging puposes, the number of nodes and edges is output for both objects

In [ ]:
# we use the networkx object, g, to create a CX2Network object
cx2_factory = cx2.NetworkXToCX2NetworkFactory()
newnetwork = cx2_factory.get_cx2network(g)

# we demonstrate that the new CX2Network object has the same number of nodes and edges as the networkx object
print('CX2Network from networkx number of nodes: ' + str(len(newnetwork.get_nodes())) + 
      ' vs Networkx: ' + str(len(g)))
print('CX2Netowrk from networkx number of edges: ' + str(len(newnetwork.get_edges())) +
      ' vs Networkx: ' + str(g.number_of_edges()))

# we will set the name of the new network to be a modification of the original network
newnetwork.set_name(cx2_network.get_name() + ' 0 degree nodes removed')

# and also set a description for the network
newnetwork.add_network_attribute('description', value='my network with 0 degree nodes removed')


# Enter your NDEx username 

Be sure to hit enter in the field to set the value!!

In [ ]:
ndexuser = getpass.getpass()

# Enter your NDEx password 

Be sure to hit enter in the field to set the value!!

In [ ]:
ndexpassword = getpass.getpass()

# Upload your network to NDEx

In [ ]:
# Create client, be sure to replace <USERNAME> and <PASSWORD> with NDEx username & password
client = ndex2.client.Ndex2(username=ndexuser, password=ndexpassword)

# Save network to NDEx, value returned is link to raw CX2 data on server.
res = client.save_new_cx2_network(newnetwork.to_cx2(), visibility='PUBLIC')

# res contains the URL to the raw network in CX2 format
# to get UUID keep string right of last / character
newnetwork_uuid = res.split('/')[-1]

print('Network UUID: ' + newnetwork_uuid)
print('Link to view network in NDEx: ' + re.sub('\/v3\/networks\/', '/viewer/networks/', res))

# Apply the style from the original network to your network

When converting to/from networkx Graph() the style on the original network is lost. To recover that style, we will use get the style from the original network using the `get_visual_properties()` method.  Then we will use `set_visual_properties()` that applies it to the current network. 

In [ ]:
vis_prop = cx2_network.get_visual_properties()
newnetwork.set_visual_properties(vis_prop)

# Update our network on NDEx

Rather then uploading an additional network to NDEx, we will update the one we just wrote using the `update_cx2_network()` method. This method requires the CX2 network as a bytes stream and the network's UUID as the second argument.

In [ ]:
# Create bytes stream
cx_stream = io.BytesIO(json.dumps(newnetwork.to_cx2(),
                                  cls=DecimalEncoder).encode('utf-8'))

# Update network in NDEx by completely replacing the network with
# one set in cx_stream
client.update_cx2_network(cx_stream, newnetwork_uuid)